In [6]:
!rm -r images
!rm -r masks

In [1]:
%load_ext autoreload
%autoreload 2

import cv2
import torch
import torch.optim as optim
from torch.utils.data import  DataLoader

import segmentation_models_pytorch as smp
import albumentations as A

from config import CFG
from Dataset import BuildDatasetParts
from scripts.Models.utils import set_seed
from scripts.Models.Trainer import Trainer
from scripts.Models.Dataset import BuildDataset
from scripts.Models.ModelBuilder import ModelBuilder
from scripts.Models.Metrics import dice_coef, iou_coef

In [2]:
def prepare_loaders():
    print("Video processing....")

    print("*********  Creating datasets  *********")
    train_dataset = BuildDatasetParts(data_transforms["train"], CFG)
    valid_dataset = BuildDatasetParts(data_transforms["valid"], CFG)
    print("*********  Creating datasets DONE  *********")

    print("********* Creating data loaders *********")
    train_dataloader = DataLoader(train_dataset, batch_size=CFG.train_bs,
                                  num_workers=2, shuffle=True, pin_memory=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=CFG.train_bs,
                                  num_workers=2, shuffle=False, pin_memory=True)
    print("********* Creating data loaders DONE *********")

    return train_dataset, valid_dataset, train_dataloader, valid_dataloader

In [3]:
set_seed()

In [4]:
data_transforms = {
    "train": A.Compose([
#                         A.HorizontalFlip(p=0.5),
#                         A.VerticalFlip(p=0.5),
                        # A.Resize(*CFG.img_size, interpolation=cv2.INTER_NEAREST),
                        A.Normalize(),
                        A.PadIfNeeded(CFG.img_size[0], CFG.img_size[1],
                                                      position = A.transforms.PadIfNeeded.PositionType.BOTTOM_RIGHT,
                                                      border_mode = cv2.BORDER_CONSTANT,
                                                      value=0, mask_value=0),
                        A.Affine(translate_percent=(0.1, 0.1)),
                        ], p=1.0),
    
    "valid": A.Compose([
        A.Resize(*CFG.img_size, interpolation=cv2.INTER_NEAREST),
        A.Normalize()
        ], p=1.0)
}

In [5]:
train_dataset, valid_dataset, train_dataloader, valid_dataloader = prepare_loaders()

Video processing....
*********  Creating datasets  *********
*********  Creating datasets DONE  *********
********* Creating data loaders *********
********* Creating data loaders DONE *********


In [ ]:
BuildDataset.show_batch_examples(train_dataset, 3)

In [8]:
model_builder = ModelBuilder(CFG, use_my_model=True, pretrained_model_path=None)

model_pytorch = model_builder.get_model()

In [9]:
optimizer = optim.Adam(model_pytorch.parameters(), lr=CFG.lr, weight_decay=CFG.wd)

def criterion(y_pred, y_true):
    return 0.5*BCELoss(y_pred, y_true) + 0.5*TverskyLoss(y_pred, y_true)

BCELoss = smp.losses.SoftBCEWithLogitsLoss()
TverskyLoss = smp.losses.TverskyLoss(mode='multilabel', log_loss=False)

loss_fn = criterion
dice_loss = dice_coef
iou_loss  = iou_coef

# Run Training

In [10]:
model_trainer = Trainer(model_pytorch, train_dataloader, valid_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        dice_loss=dice_loss,
                        iou_loss=iou_loss,
                        config=CFG)

In [ ]:
model_trainer.run_training()

Epoch 1/50


Train :   0%|          | 0/15 [00:00<?, ?it/s]/home/raphael/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


# Result

In [4]:
model_trainer.plot_logs()

In [5]:
model_trainer.show_result(valid_dataset, 6)